# Tracking 

Two important processes in normal tissue development and disease are cell migration and proliferation. To gain a better understanding on these processes, tracking in time-lapse datasets is needed. Tracking is the motion-analysis of individual objects over space and time. Hereby a unique ID for each detected object is generated and maintained.

To find out more about tracking, we will explore a cancer cell migration dataset from [Tinevez, J. & Guillaume Jacquemet, G.](https://zenodo.org/record/5206107#.ZFthHnZBxPa) licensed by [CC BY 4.0](https://creativecommons.org/licenses/by/4.0/legalcode). We will concentrate on a [cropped](https://haesleinhuepf.github.io/BioImageAnalysisNotebooks/12_image_analysis_basics/04_Cropping_images.html) region of the dataset (in x, y and t).

In [23]:
import btrack
from skimage.io import imread, imsave
import napari
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from napari_skimage_regionprops import add_table

First, we need to read in our segmentation result as it is the base of the tracking. Let's also read in the image itself, to be able to see both even though it's not needed for the tracking itself.

In [2]:
image = imread('../../data/cancer_cell_migration_cropped.tif')
label_image = imread('../../data/cancer_cell_migration_voronoi_otsu_labeling_cropped.tif')

We can check our image `shape` 

In [3]:
label_image.shape

(48, 1, 130, 130)

In [4]:
image = image[:,0,...] #Remove Second Dim
label_image = label_image[:,0,...] #Remove Second Dim

In [5]:
label_image.shape

(48, 130, 130)

And see it sorted with the dimensions `[t,z,y,x]` with
- `t` = time
- `z` = number of z-slices
- `y` = number of pixels in y
- `x` = number of pixels in x

We can use [`stackview`](https://github.com/haesleinhuepf/stackview) to visualize this 4D stack.

In [6]:
import stackview

`stackview.curtain` allows us to visualize our label_image on top of our image

In [7]:
stackview.curtain(image, label_image, continuous_update=True,zoom_factor = 5) 

In [8]:
from napari_skimage_regionprops import regionprops_table_all_frames

In [20]:
stats = regionprops_table_all_frames(image[:,np.newaxis,:,:], 
                             label_image[:,np.newaxis,:,:],
                             size=True)
table = pd.DataFrame(stats)

analyzing frame 0
analyzing frame 1
analyzing frame 2
analyzing frame 3
analyzing frame 4
analyzing frame 5
analyzing frame 6
analyzing frame 7
analyzing frame 8
analyzing frame 9
analyzing frame 10
analyzing frame 11
analyzing frame 12
analyzing frame 13
analyzing frame 14
analyzing frame 15
analyzing frame 16
analyzing frame 17
analyzing frame 18
analyzing frame 19
analyzing frame 20
analyzing frame 21
analyzing frame 22
analyzing frame 23
analyzing frame 24
analyzing frame 25
analyzing frame 26
analyzing frame 27
analyzing frame 28
analyzing frame 29
analyzing frame 30
analyzing frame 31
analyzing frame 32
analyzing frame 33
analyzing frame 34
analyzing frame 35
analyzing frame 36
analyzing frame 37
analyzing frame 38
analyzing frame 39
analyzing frame 40
analyzing frame 41
analyzing frame 42
analyzing frame 43
analyzing frame 44
analyzing frame 45
analyzing frame 46
analyzing frame 47


In [8]:
# create btrack objects (with properties) from the segmentation data
# (you can also calculate properties, based on scikit-image regionprops)
FEATURES = [
    "area"
]

# _Substitute the next thing_

In [9]:
# connect labels to properties
objects = btrack.utils.segmentation_to_objects(
    label_image, 
    properties=tuple(FEATURES)
)

[INFO][2023/05/22 04:26:30 PM] Localizing objects from segmentation...
[INFO][2023/05/22 04:26:31 PM] Objects are of type: <class 'dict'>
[INFO][2023/05/22 04:26:31 PM] ...Found 650 objects in 48 frames.


In [14]:
# connect labels to properties
objects = btrack.utils.segmentation_to_objects(
    label_image
)

[INFO][2023/05/22 04:42:16 PM] Localizing objects from segmentation...
[INFO][2023/05/22 04:42:16 PM] Objects are of type: <class 'dict'>
[INFO][2023/05/22 04:42:16 PM] ...Found 650 objects in 48 frames.


In [15]:
from btrack import datasets

In [16]:
config_file = datasets.cell_config()

In [17]:
config_file

'C:\\Users\\maral\\AppData\\Local\\btrack-examples\\btrack-examples\\Cache\\examples\\cell_config.json'

In [18]:
# initialise a tracker session using a context manager
with btrack.BayesianTracker() as tracker:

  # configure the tracker using a preconfigured config file
  tracker.configure(config_file)

  # append the objects to be tracked
  tracker.append(objects)
    
  # set the volume (Z axis volume limits default to [-1e5, 1e5] for 2D data)
  tracker.volume = ((0, 130), (0, 130)) # when having 3D data this needs to be modified to X,Y and Z range

  # track
  tracker.track_interactive(step_size=100)

  # get the tracks in a format for napari visualization
  data, properties, graph = tracker.to_napari(ndim=2)

[INFO][2023/05/22 04:42:18 PM] Loaded btrack: C:\Users\maral\mambaforge\envs\laptrack_env2\lib\site-packages\btrack\libs\libtracker.DLL
[INFO][2023/05/22 04:42:18 PM] Starting BayesianTracker session
[INFO][2023/05/22 04:42:18 PM] Loading configuration file: C:\Users\maral\AppData\Local\btrack-examples\btrack-examples\Cache\examples\cell_config.json
[INFO][2023/05/22 04:42:18 PM] Objects are of type: <class 'list'>
[WARNING][2023/05/22 04:42:18 PM] `track_interactive` will be deprecated. Use `track` instead.
[INFO][2023/05/22 04:42:18 PM] Starting tracking... 
[INFO][2023/05/22 04:42:18 PM] Update using: ['MOTION']
[INFO][2023/05/22 04:42:18 PM] Tracking objects in frames 0 to 48 (of 48)...
[INFO][2023/05/22 04:42:18 PM]  - Timing (Bayesian updates: 0.00ms, Linking: 0.00ms)
[INFO][2023/05/22 04:42:18 PM]  - Probabilities (Link: 0.99998, Lost: 0.46137)
[INFO][2023/05/22 04:42:18 PM] SUCCESS.
[INFO][2023/05/22 04:42:18 PM]  - Found 30 tracks in 48 frames (in 0.0s)
[INFO][2023/05/22 04:42

# _It would be cool to do this after matching labels somehow between the frames_

In [25]:
viewer = napari.Viewer()
labels_layer = viewer.add_labels(label_image, features = table)
tracks_layer = viewer.add_tracks(data, properties=properties, graph=graph)
add_table(labels_layer, viewer)

Napari status bar display of label properties disabled because https://github.com/napari/napari/issues/5417 and https://github.com/napari/napari/issues/4342


- https://www.youtube.com/watch?v=JZ6QUZIds2g